In [1]:
import boto3
import zipfile
import os

# Especificar o perfil do AWS CLI
session = boto3.Session(profile_name='CRISTIAN_AWS')

# Criar cliente do Lambda com a sessão específica
lambda_client = session.client('lambda')

# Função para criar um arquivo zip contendo o código da Lambda
def criar_arquivo_zip():
    # Código da função Lambda
    codigo_lambda = """
import json
import boto3
import random

# Conectar ao DynamoDB
dynamodb = boto3.resource('dynamodb')
tabela = dynamodb.Table('PerguntasSimulado')

def lambda_handler(event, context):
    # Obter todas as perguntas do DynamoDB
    response = tabela.scan()
    perguntas = response['Items']
    
    # Selecionar uma pergunta aleatória
    pergunta_escolhida = random.choice(perguntas)
    
    return {
        'statusCode': 200,
        'body': json.dumps(pergunta_escolhida)
    }
    """

    # Criar um arquivo ZIP contendo o código da Lambda
    with open("lambda_function.py", "w") as arquivo_lambda:
        arquivo_lambda.write(codigo_lambda)
    
    # Compactar o arquivo
    with zipfile.ZipFile("lambda_function.zip", "w") as zipf:
        zipf.write("lambda_function.py")

# Função para criar a Lambda na AWS
def criar_lambda():
    criar_arquivo_zip()
    
    # Ler o arquivo ZIP criado
    with open("lambda_function.zip", 'rb') as f:
        codigo_zip = f.read()

    try:
        response = lambda_client.create_function(
            FunctionName='SimuladoAWSFunction',
            Runtime='python3.9',
            Role='arn:aws:iam::058264285974:role/LambdaDynamoDBRole',  # Coloque o ARN da IAM Role que a Lambda vai usar
            Handler='lambda_function.lambda_handler',
            Code={'ZipFile': codigo_zip},
            Description='Função Lambda para pegar perguntas do DynamoDB',
            Timeout=15,
            MemorySize=128,
            Publish=True
        )
        print("Função Lambda criada com sucesso!")
        print(response)
    except Exception as e:
        print(f"Erro ao criar a função Lambda: {e}")

# Chamar a função para criar a Lambda
criar_lambda()

Função Lambda criada com sucesso!
{'ResponseMetadata': {'RequestId': '4437e527-38d1-42d4-974f-0dfc5909a82b', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Wed, 09 Oct 2024 03:43:18 GMT', 'content-type': 'application/json', 'content-length': '1388', 'connection': 'keep-alive', 'x-amzn-requestid': '4437e527-38d1-42d4-974f-0dfc5909a82b'}, 'RetryAttempts': 0}, 'FunctionName': 'SimuladoAWSFunction', 'FunctionArn': 'arn:aws:lambda:us-east-1:058264285974:function:SimuladoAWSFunction', 'Runtime': 'python3.9', 'Role': 'arn:aws:iam::058264285974:role/LambdaDynamoDBRole', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 642, 'Description': 'Função Lambda para pegar perguntas do DynamoDB', 'Timeout': 15, 'MemorySize': 128, 'LastModified': '2024-10-09T03:43:17.754+0000', 'CodeSha256': 'xd4y1HeRMeNn7+NIPOU8Uclxbk+w3aEptb5y9XBwp8k=', 'Version': '1', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '2bac12b3-8a2e-468b-a3da-e2d44a6c0ae6', 'State': 'Pending', 'StateReason': 'The functio